In [2]:
import os
from dotenv import load_dotenv
import psycopg
from psycopg.rows import dict_row

load_dotenv()
DB_URL = os.getenv("DATABASE_URL")
if not DB_URL:
    raise SystemExit("Set DATABASE_URL in environment or .env")

import os, socket, urllib.parse as up

if not os.getenv("PGHOSTADDR"):
    host = up.urlparse(DB_URL).hostname
    ipv4 = next(ai[4][0] for ai in socket.getaddrinfo(host, 5432, family=socket.AF_INET))
    os.environ["PGHOSTADDR"] = ipv4
    os.environ["PGCONNECT_TIMEOUT"] = "10"
    print("Using IPv4:", ipv4)

def connect():
    # No 'options=...' here; Neon pooler rejects startup params
    return psycopg.connect(
        DB_URL,
        row_factory=dict_row,
        connect_timeout=10,
        sslmode="require",
    )
def set_session(cur):
    # Set per-session params after connecting
    cur.execute("set statement_timeout = '15s'")
    cur.execute("set application_name = 'smoke_test'")

def main():
    print("Connecting...")
    with connect() as conn, conn.cursor() as cur:
        set_session(cur)  # apply timeout/app name
        # Basic table counts
        cur.execute("select count(*) as n from perigon.articles")
        print("perigon.articles count:", cur.fetchone()["n"])
        cur.execute("select task, count(*) as n from perigon.article_summary_annotations group by task order by task")
        print("article_summary_annotations by task:", cur.fetchall())

        # How many frames rows actually have a non-empty parsed array?
        cur.execute("""
            select count(*) as frames_rows,
                   count(*) filter (where jsonb_array_length(annotation_parsed) > 0) as frames_with_items
            from perigon.article_summary_annotations
            where task='frames'
        """)
        print("frames rows:", cur.fetchone())

        # How many meso rows with items?
        cur.execute("""
            select count(*) as meso_rows,
                   count(*) filter (where jsonb_typeof(annotation_parsed)='array' and jsonb_array_length(annotation_parsed) > 0) as meso_with_items
            from perigon.article_summary_annotations
            where task='meso'
        """)
        print("meso rows:", cur.fetchone())

        # Frames that also have an explicit RELEVANT relevance row
        cur.execute("""
            select count(*) as frames_with_rel
            from perigon.article_summary_annotations f
            where f.task='frames'
              and exists (
                select 1
                from perigon.article_summary_annotations r
                where r.article_id=f.article_id
                  and r.task='relevance'
                  and coalesce(r.annotation_parsed->>'relevance','')='RELEVANT'
              )
        """)
        print("frames with explicit RELEVANT:", cur.fetchone()["frames_with_rel"])

        # Sample annotated articles (latest by pub_date)
        cur.execute("""
            select a.article_id, a.source_domain, a.pub_date::timestamp as pub_date
            from perigon.articles a
            where exists (
              select 1 from perigon.article_summary_annotations x
              where x.article_id=a.article_id and x.task in ('frames','meso')
            )
            order by a.pub_date desc nulls last
            limit 5
        """)
        print("\nSample annotated articles (by pub_date):")
        for row in cur.fetchall():
            print(row)

        # Sample frames entries (show first frame text if present)
        cur.execute("""
            with fr as (
              select f.article_id,
                     case when jsonb_typeof(f.annotation_parsed)='array' and jsonb_array_length(f.annotation_parsed)>0
                          then (f.annotation_parsed->>0) else null end as first_frame,
                     jsonb_array_length(f.annotation_parsed) as n_frames
              from perigon.article_summary_annotations f
              where f.task='frames'
            )
            select fr.article_id, fr.n_frames, fr.first_frame
            from fr
            order by fr.n_frames desc nulls last
            limit 5
        """)
        print("\nSample frames rows:")
        for row in cur.fetchall():
            print(row)

        # Sample meso entries (show first object keys)
        cur.execute("""
            with me as (
              select m.article_id,
                     case when jsonb_typeof(m.annotation_parsed)='array' and jsonb_array_length(m.annotation_parsed)>0
                          then m.annotation_parsed->0 else null end as first_obj,
                     jsonb_array_length(m.annotation_parsed) as n_meso
              from perigon.article_summary_annotations m
              where m.task='meso'
            )
            select article_id, n_meso,
                   first_obj->>'narrative frame' as narrative_frame,
                   first_obj->>'meso narrative' as meso_narrative
            from me
            order by n_meso desc nulls last
            limit 5
        """)
        print("\nSample meso rows:")
        for row in cur.fetchall():
            print(row)

if __name__ == "__main__":
    main()

Using IPv4: 13.43.29.36
Connecting...
perigon.articles count: 1431
article_summary_annotations by task: [{'task': 'frames', 'n': 644}, {'task': 'meso', 'n': 628}, {'task': 'relevance', 'n': 1266}]
frames rows: {'frames_rows': 644, 'frames_with_items': 628}
meso rows: {'meso_rows': 628, 'meso_with_items': 616}
frames with explicit RELEVANT: 644

Sample annotated articles (by pub_date):
{'article_id': '05b7e2e1f5684e589ed28d324d18688f', 'source_domain': 'theguardian.com', 'pub_date': datetime.datetime(2025, 9, 10, 22, 6, 39)}
{'article_id': 'fdf62547c6334972bc3f28d4506852ad', 'source_domain': 'bbc.co.uk', 'pub_date': datetime.datetime(2025, 9, 10, 22, 5, 3, 755000)}
{'article_id': '0f108cced29e47d6b7079be53e676306', 'source_domain': 'bbc.co.uk', 'pub_date': datetime.datetime(2025, 9, 10, 21, 50, 41)}
{'article_id': '982d4d828f5b4332b8d297a64e5c0bdf', 'source_domain': 'bbc.co.uk', 'pub_date': datetime.datetime(2025, 9, 10, 20, 46, 52, 236000)}
{'article_id': 'c7cb319da538413a9d5675affd9d7

In [3]:
import pandas as pd

def df_from_query(sql: str, params=()):
    with connect() as conn, conn.cursor(row_factory=dict_row) as cur:
        cur.execute(sql, params)
        rows = cur.fetchall()
    return pd.DataFrame(rows)

def fetch_sample_data():
    # Latest articles (whether annotated or not)
    articles_df = df_from_query("""
        select a.article_id, a.source_domain, a.pub_date::timestamp as pub_date
        from perigon.articles a
        order by a.pub_date desc nulls last
        limit 10
    """)

    # Relevance rows (show parsed relevance + raw snippet)
    relevance_df = df_from_query("""
        select
          a.article_id,
          a.pub_date::timestamp as pub_date,
          r.annotation_parsed->>'relevance' as relevance,
          left(r.annotation_raw_json::text, 160) as raw_snippet
        from perigon.articles a
        join perigon.article_summary_annotations r
          on r.article_id = a.article_id
         and r.task = 'relevance'
        order by a.pub_date desc nulls last
        limit 10
    """)

    # Frames (one row per frame value)
    frames_df = df_from_query("""
        select
          a.article_id,
          a.pub_date::timestamp as pub_date,
          fr.frame as frame
        from perigon.articles a
        join perigon.article_summary_annotations f
          on f.article_id = a.article_id
         and f.task = 'frames'
         and jsonb_typeof(f.annotation_parsed) = 'array'
         and jsonb_array_length(f.annotation_parsed) > 0
        cross join lateral jsonb_array_elements_text(f.annotation_parsed) as fr(frame)
        order by a.pub_date desc nulls last
        limit 25
    """)

    # Meso narratives (one row per meso object)
    meso_df = df_from_query("""
        select
          a.article_id,
          a.pub_date::timestamp as pub_date,
          e.value->>'narrative frame' as narrative_frame,
          e.value->>'meso narrative'     as meso_narrative
        from perigon.articles a
        join perigon.article_summary_annotations m
          on m.article_id = a.article_id
         and m.task = 'meso'
         and jsonb_typeof(m.annotation_parsed) = 'array'
         and jsonb_array_length(m.annotation_parsed) > 0
        cross join lateral jsonb_array_elements(m.annotation_parsed) as e(value)
        order by a.pub_date desc nulls last
        limit 25
    """)

    return articles_df, relevance_df, frames_df, meso_df

# Run and display
articles_df, relevance_df, frames_df, meso_df = fetch_sample_data()
print("Articles:", articles_df.shape); display(articles_df.head(10))
print("Relevance:", relevance_df.shape); display(relevance_df.head(10))
print("Frames:", frames_df.shape); display(frames_df.head(25))
print("Meso:", meso_df.shape); display(meso_df.head(25))

Articles: (10, 3)


,article_id,source_domain,pub_date
0,05b7e2e1f5684e589ed28d324d18688f,theguardian.com,2025-09-10 22:06:39.000
1,fdf62547c6334972bc3f28d4506852ad,bbc.co.uk,2025-09-10 22:05:03.755
2,0f108cced29e47d6b7079be53e676306,bbc.co.uk,2025-09-10 21:50:41.000
3,982d4d828f5b4332b8d297a64e5c0bdf,bbc.co.uk,2025-09-10 20:46:52.236
4,71cf1e02548a44c78fc745e2a56e8681,telegraph.co.uk,2025-09-10 18:59:00.000
5,5bebf4fbc63d432abb33c9535163f632,theguardian.com,2025-09-10 15:33:10.000
6,c7cb319da538413a9d5675affd9d77c1,theguardian.com,2025-09-10 15:28:36.000
7,c551d7528907413f8c32f49b40aa1dab,theguardian.com,2025-09-10 15:27:36.000
8,78c0c18d552b4690b37e2dd0cb1994e9,telegraph.co.uk,2025-09-10 15:20:00.000
9,bd2115342ef84f1abffe9d93744929c0,theguardian.com,2025-09-10 15:08:16.000


Relevance: (10, 4)


,article_id,pub_date,relevance,raw_snippet
0,05b7e2e1f5684e589ed28d324d18688f,2025-09-10 22:06:39.000,RELEVANT,"""```json\n{\""relevance\"":\""RELEVANT\""}\n```<|i..."
1,fdf62547c6334972bc3f28d4506852ad,2025-09-10 22:05:03.755,RELEVANT,"""```json\n{\""relevance\"":\""RELEVANT\""}\n```<|i..."
2,0f108cced29e47d6b7079be53e676306,2025-09-10 21:50:41.000,RELEVANT,"""```json\n{\""relevance\"":\""RELEVANT\""}\n```<|i..."
3,982d4d828f5b4332b8d297a64e5c0bdf,2025-09-10 20:46:52.236,RELEVANT,"""```json\n{\""relevance\"":\""RELEVANT\""}\n```<|i..."
4,5bebf4fbc63d432abb33c9535163f632,2025-09-10 15:33:10.000,IRRELEVANT,"""{\""relevance\"":\""IRRELEVANT\""}<|im_end|>"""
5,c7cb319da538413a9d5675affd9d77c1,2025-09-10 15:28:36.000,RELEVANT,"""```json\n{\""relevance\"":\""RELEVANT\""}\n```<|i..."
6,c551d7528907413f8c32f49b40aa1dab,2025-09-10 15:27:36.000,RELEVANT,"""```json\n{\""relevance\"":\""RELEVANT\""}\n```<|i..."
7,bd2115342ef84f1abffe9d93744929c0,2025-09-10 15:08:16.000,IRRELEVANT,"""{\""relevance\"":\""IRRELEVANT\""}<|im_end|>"""
8,300e904cd6a54fc99d6b41d7817dabd3,2025-09-10 15:00:07.000,IRRELEVANT,"""```json\n{\""relevance\"":\""IRRELEVANT\""}\n```<..."
9,91cc3d5a922648dba4bc484a67af8428,2025-09-10 14:30:54.000,IRRELEVANT,"""```json\n{\""relevance\"":\""IRRELEVANT\""}\n```<..."


Frames: (25, 3)


,article_id,pub_date,frame
0,05b7e2e1f5684e589ed28d324d18688f,2025-09-10 22:06:39.000,anti-immigration policies
1,fdf62547c6334972bc3f28d4506852ad,2025-09-10 22:05:03.755,allocation of resources to migrants
2,fdf62547c6334972bc3f28d4506852ad,2025-09-10 22:05:03.755,asylum seekers
3,fdf62547c6334972bc3f28d4506852ad,2025-09-10 22:05:03.755,cultural impacts of migration
4,fdf62547c6334972bc3f28d4506852ad,2025-09-10 22:05:03.755,public opinion on migration
5,982d4d828f5b4332b8d297a64e5c0bdf,2025-09-10 20:46:52.236,allocation of resources to migrants
6,982d4d828f5b4332b8d297a64e5c0bdf,2025-09-10 20:46:52.236,migrants' access to justice system
7,982d4d828f5b4332b8d297a64e5c0bdf,2025-09-10 20:46:52.236,asylum seekers
8,c7cb319da538413a9d5675affd9d77c1,2025-09-10 15:28:36.000,racism or xenophobia toward migrants
9,c7cb319da538413a9d5675affd9d77c1,2025-09-10 15:28:36.000,political scapegoating of migrants


Meso: (25, 4)


,article_id,pub_date,narrative_frame,meso_narrative
0,05b7e2e1f5684e589ed28d324d18688f,2025-09-10 22:06:39.000,anti-immigration policies,Supporting anti-immigration policies through m...
1,fdf62547c6334972bc3f28d4506852ad,2025-09-10 22:05:03.755,public opinion on migration,Public opinion is divided or mixed
2,fdf62547c6334972bc3f28d4506852ad,2025-09-10 22:05:03.755,cultural impacts of migration,Migration involves cultural and religious diff...
3,fdf62547c6334972bc3f28d4506852ad,2025-09-10 22:05:03.755,allocation of resources to migrants,Aid to migrants diverts resources from local c...
4,fdf62547c6334972bc3f28d4506852ad,2025-09-10 22:05:03.755,public opinion on migration,Public opinion supports stricter migration limits
5,fdf62547c6334972bc3f28d4506852ad,2025-09-10 22:05:03.755,cultural impacts of migration,Migration leads to mental and social differenc...
6,fdf62547c6334972bc3f28d4506852ad,2025-09-10 22:05:03.755,asylum seekers,Asylum seekers come from diverse countries and...
7,982d4d828f5b4332b8d297a64e5c0bdf,2025-09-10 20:46:52.236,asylum seekers,Asylum seekers are housed in temporary or non-...
8,c7cb319da538413a9d5675affd9d77c1,2025-09-10 15:28:36.000,political scapegoating of migrants,Politicians unfairly blame migrants for societ...
9,c7cb319da538413a9d5675affd9d77c1,2025-09-10 15:28:36.000,racism or xenophobia toward migrants,Xenophobia harms social cohesion and integration
